In [95]:
import pandas as pd
import numpy  as np
import matplotlib as plt

In [116]:
df1 = pd.read_csv('./Teste-30-09.csv')
df1 = df1[['compute_version','registers','smem','cmem','num_of_cores','L2','bandwith','theoretical_flops','AppId','number_of_lines_kernel','occupancy','input_size','duration','block_x']]
#df1 = df1[['compute_version','registers','smem','cmem','AppId','number_of_lines_kernel','occupancy','duration','block_x']]


In [117]:
#df1 = df[['AppName','compute_version','registers','smem','cmem','gpu_id','gpu_name','l1_cache_used','max_clock_rate','num_of_cores','L2','bus','memory_clock','bandwith','GM','num_sm','num_cores_sm','theoretical_flops','AppId','number_of_lines_kernel','occupancy','input_size', 'duration','grid_x','grid_y','block_x','block_y']]
#df1 = df[['AppName','compute_version','registers','smem','cmem','gpu_id','gpu_name','max_clock_rate','num_of_cores','L2','bus','memory_clock','bandwith','GM','num_sm','num_cores_sm','theoretical_flops','AppId','number_of_lines_kernel','occupancy','duration','grid_x','grid_y','block_x','block_y']]

df1 = df1.sort_values(by='AppId', ascending=True)
#df1['duration'] = pd.to_numeric(df1['duration'])
#df1[['registers','smem','cmem','gpu_id','max_clock_rate','num_of_cores','bus','memory_clock','GM','num_sm','num_cores_sm','AppId','number_of_lines_kernel','occupancy','duration','grid_x','grid_y','block_x','block_y']] = df1[['registers','smem','cmem','gpu_id','max_clock_rate','num_of_cores','bus','memory_clock','GM','num_sm','num_cores_sm','AppId','number_of_lines_kernel','occupancy','duration','grid_x','grid_y','block_x','block_y']].astype(float)
df1 = df1.select_dtypes(exclude=['object'])
#df1 = df1.drop(columns= ["Unnamed: 0"], axis = 1)
#df1.to_csv('testando.csv')
#df1=df1.dropna()
#df1.dtypes


In [113]:
from sklearn import metrics

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [137]:
from sklearn.ensemble import RandomForestRegressor

for idApp in range(6,7):
  idApp = float(idApp)
  
  X_train = df1[df1['AppId'] != idApp]
  Y_train = df1[df1['AppId'] != idApp]['duration']
  X_test = df1[df1['AppId'] == idApp]
  Y_test = df1[df1['AppId'] == idApp]['duration']
  
  yTrain = np.log(Y_train + 0.1)
  yTest = np.log(Y_test + 0.1)
  #xTrain = np.log(X_train + 0.000001)
  #xTest = np.log(X_test + 0.000001)

  regressor = RandomForestRegressor(n_estimators = 100, random_state =42)
  regressor.fit(X_train, yTrain)

  y_pred = regressor.predict(X_test)
  #print(regressor.score())

  #df2=pd.DataFrame({'Actual':yTest, 'Predicted':y_pred})
  #tt=pd.Series({'Actual':yTest, 'Predicted':y_pred, 'X_Test':X_test})

  df2 = pd.DataFrame(X_test)
  df3= pd.DataFrame({'Actual':yTest, 'Predicted':y_pred})

  print('Application: ', idApp, ' Percentage:', mean_absolute_percentage_error(yTest, y_pred))
  

Application:  6.0  Percentage: 0.0010201854750769442


In [138]:
df5 = df1[df1['AppId'] == 6]
df5 = pd.concat([df5, df3], axis=1)
df5.to_csv('AppId-6.csv', sep = ',')

Application:  1.0  Percentage: 4.030940236425199
Application:  2.0  Percentage: 0.6207321530121436
Application:  3.0  Percentage: 0.3365748131309271
Application:  4.0  Percentage: 0.3939820199634534
Application:  5.0  Percentage: 0.0028232029532914735
Application:  6.0  Percentage: 0.0008928604112268108
Application:  7.0  Percentage: 0.008234390935475972
Application:  8.0  Percentage: 0.002211841244878032
Application:  9.0  Percentage: 0.033268159684096875

Application:  1.0  Percentage: 3.6619987773468985
Application:  2.0  Percentage: 0.25954225973794276
Application:  3.0  Percentage: 0.30439375781936434
Application:  4.0  Percentage: 1.2143470939297427
Application:  5.0  Percentage: 0.011393695476755894
Application:  6.0  Percentage: 0.012798110210881595
Application:  7.0  Percentage: 0.02175096369648205
Application:  8.0  Percentage: 0.01229895040559069
Application:  9.0  Percentage: 0.051949570986811223

100:
Application:  1.0  Percentage: 2.4568160831735066
Application:  2.0  Percentage: 0.35518768529248285
Application:  3.0  Percentage: 0.5822789950166065
Application:  4.0  Percentage: 7.949959177626566
Application:  5.0  Percentage: 0.03537103201123657
Application:  6.0  Percentage: 0.047906109856315174
Application:  7.0  Percentage: 0.030826298896368093
Application:  8.0  Percentage: 0.025104629924483625
Application:  9.0  Percentage: 0.16431058776644525